In [ ]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install --no-deps packaging ninja einops flash-attn trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
import torch
import os
from google.colab import userdata
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,GemmaTokenizer
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
import os
os.environ["HF_TOKEN"]=userdata.get("HF_TOKEN")

In [ ]:
model_id="google/gemma-2b"
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_id,use_auth_token=True)

In [ ]:
model=AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    token=os.environ["HF_TOKEN"]
)

In [ ]:
text="Quote: imagination is more,"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model.generate(**input,max_new_tokens=50)

In [ ]:
answer=tokenizer.decode(output[0],skip_special_tokens=True)

In [ ]:
answer

In [ ]:
os.environ["WANDB_DISABLED"]="false"
lora_config=LoraConfig(
    r=8,
    target_modules=["q_proj","o_proj","k_proj","v_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM"
)

In [ ]:
from datasets import load_dataset
data=load_dataset("Abirate/english_quotes")
data=data.map(lambda samples:tokenizer(samples["quote"]),batched=True)

In [ ]:
def format_func(example):
  text=f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
  return [text]

In [ ]:
import transformers
trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=format_func
)

In [ ]:
trainer.train()

In [ ]:
text="Quote: a women like a teabag,"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model.generate(**input,max_new_tokens=50)

In [ ]:
print(tokenizer.decode(output[0],skip_special_tokens=True))